In [1]:
import pandas as pd

soi = ['kluyveromyces_marxianus', 'komagataella_phaffii', 'yarrowia_lipolytica']

df = pd.read_csv('data/input/fourdbs_hi_input_species.csv')
# df[~df['species_name'].isin(soi)].reset_index(drop=True).to_csv('data/input/fourdbs_hi_without_3soi.csv', index=False)
# df[df['species_name'].isin(soi)].reset_index(drop=True).to_csv('data/input/3soi.csv', index=False)

run allegro on without_3soi from above

In [3]:
import re
from Bio import SeqIO

from src.utils.guide_finder import GuideFinder


def find_all_matches(list1, list2):
    matches = list()

    for index, item in enumerate(list2):
        if item in list1:
            matches.append(index)
        
    return matches

def hamming_distance(str1: str, str2: str, length: int) -> int:
    """
    Calculates the Hamming distance between two strings up to a specified length.
    """
    if len(str1) < length or len(str2) < length:
        raise ValueError('Strings must have a length of at least {n}'.format(n=length))

    return sum(ch1 != ch2 for ch1, ch2 in zip(str1[:length], str2[:length]))


def get_record_metadata(record):
    # This gene's own name -- Usually N/A for unannotated CDS files or chromosomes/scaffolds
    gene_match = re.search(gene_regex, record.description)
    gene_name = gene_match.group(1) if gene_match is not None else 'N/A'

    # This gene's own protein id e.g., XP_022674739.1
    protein_id_match = re.search(protein_id_regex, record.description)
    protein_id = protein_id_match.group(1) if protein_id_match is not None else 'N/A'

    # For example, [orthologous_to_ref_protein=XP_022674739.1], extracts XP_022674739.1
    ortho_prot_to_match = re.search(orthologous_protein_regex, record.description)
    ortho_prot_id = ortho_prot_to_match.group(1) if ortho_prot_to_match is not None else 'N/A'

    # For example, [orthologous_to_gene=HIS7], extracts HIS7
    ortho_gene_to_match = re.search(orthologous_name_regex, record.description)
    ortho_gene_name = ortho_gene_to_match.group(1) if ortho_gene_to_match is not None else 'N/A'

    return gene_name, protein_id, ortho_prot_id, ortho_gene_name


gene_regex = r'\[gene=(.*?)\]'
tag_regex = r'\[locus_tag=(.*?)\]'
protein_id_regex = r'\[protein_id=(.*?)\]'
reference_species_regex = r'\[ref_species=(.*?)\]'
orthologous_name_regex = r'\[orthologous_to_gene=(.*?)\]'
orthologous_protein_regex = r'\[orthologous_to_ref_protein=(.*?)\]'

new_exp_name = 'all_without_threeSOI_a7'

split_out = pd.read_csv(f'data/output/{new_exp_name}/{new_exp_name}.csv')

library = split_out.sequence.unique()

test_df = pd.read_csv(f'data/input/3soi.csv')[['species_name', 'cds_file_name']]

df_species_name_list = list()
df_covered_list = list()
df_strands = list()
df_mismatch = list()
df_locations = list()
df_gene_names = list()
df_protein_ids = list()
df_ortho_prot_ids = list()
df_ortho_gene_names = list()
df_lib_partial_match = list()

gf = GuideFinder()
base_path = 'data/input/cds/orthogroups/'

for label, row in test_df.iterrows():
    species_name = row['species_name']

    species_is_covered_n_times = 0
    exact_matching_guides = list()

    records_path = base_path + row['cds_file_name']
    records = list(SeqIO.parse(open(records_path), 'fasta'))

    for record in records:
        gene_name, protein_id, ortho_prot_id, ortho_gene_name = get_record_metadata(record)

        (guides_list,
        _guides_context_list,
        strands_list,
        locations_list) = gf.identify_guides_and_indicate_strand(
            pam='NGG',
            sequence=str(record.seq).upper(),
            protospacer_length=20,
            context_toward_five_prime=0,
            context_toward_three_prime=0,
            filter_repetitive=False
        )

        # Attempt to find an exact match.
        matches = find_all_matches(library, guides_list)
        for match in matches:
            df_species_name_list.append(species_name)
            df_covered_list.append(guides_list[match])
            df_locations.append(locations_list[match])
            df_strands.append(strands_list[match])
            df_gene_names.append(gene_name)
            df_protein_ids.append(protein_id)
            df_ortho_prot_ids.append(ortho_prot_id)
            df_ortho_gene_names.append(ortho_gene_name)
            df_lib_partial_match.append('N/A')
            df_mismatch.append('N/A')
            species_is_covered_n_times += 1
            exact_matching_guides.append(match)
    
    # Attempt to find partial matches.
    mm_allowed = 2
    req_match_len = 10

    for record in records:
        gene_name, protein_id, ortho_prot_id, ortho_gene_name = get_record_metadata(record)

        (guides_list,
        _guides_context_list,
        strands_list,
        locations_list) = gf.identify_guides_and_indicate_strand(
            pam='NGG',
            sequence=str(record.seq).upper(),
            protospacer_length=20,
            context_toward_five_prime=0,
            context_toward_three_prime=0,
            filter_repetitive=False
        )

        for idx, test_guide in enumerate(guides_list):
            for lib_guide in library:

                # do not allow double-counting exact matching guides from before.
                if lib_guide in exact_matching_guides:
                    continue

                distance_after_seed = hamming_distance(lib_guide, test_guide, req_match_len)
                
                if lib_guide[req_match_len:] == test_guide[req_match_len:] and distance_after_seed <= mm_allowed and distance_after_seed > 0:
                    df_species_name_list.append(species_name)
                    df_covered_list.append(test_guide)
                    df_locations.append(locations_list[idx])
                    df_strands.append(strands_list[idx])
                    df_gene_names.append(gene_name)
                    df_protein_ids.append(protein_id)
                    df_ortho_prot_ids.append(ortho_prot_id)
                    df_ortho_gene_names.append(ortho_gene_name)
                    df_mismatch.append(distance_after_seed)
                    df_lib_partial_match.append(lib_guide)
                    species_is_covered_n_times += 1

    # No match.
    if species_is_covered_n_times == 0:
        df_species_name_list.append(species_name)
        df_covered_list.append('Not covered')
        df_strands.append('Not covered')
        df_locations.append('Not covered')
        df_gene_names.append('Not covered')
        df_protein_ids.append('Not covered')
        df_ortho_prot_ids.append('Not covered')
        df_ortho_gene_names.append('Not covered')
        df_mismatch.append('Not covered')
        df_lib_partial_match.append('Not covered')

test_df = pd.DataFrame.from_dict({
    'species': df_species_name_list,
    'covered_by': df_covered_list,
    'partial_match_in_library': df_lib_partial_match,
    'mismatch': df_mismatch,
    'strand': df_strands,
    'location': df_locations,
    'gene_name': df_gene_names,
    'protein_id': df_protein_ids,
    'ortho_prot_id': df_ortho_prot_ids,
    'ortho_gene_name': df_ortho_gene_names,
})

test_df.to_csv(f'data/output/test_{new_exp_name}_results.csv', index=False)

In [4]:
pd.read_csv(f'data/output/test_{new_exp_name}_results.csv')

,species,covered_by,partial_match_in_library,mismatch,strand,location,gene_name,protein_id,ortho_prot_id,ortho_gene_name
0,yarrowia_lipolytica,CATCTCCAGATGATTGCCAT,NaN,NaN,F,224,NaN,XP_500977.1,NP_012965.3,GAP1
1,yarrowia_lipolytica,CTCCAGATGATTGCCATCGG,NaN,NaN,F,227,NaN,XP_500977.1,NP_012965.3,GAP1
2,yarrowia_lipolytica,ATGACGACGTTCATGACAGA,NaN,NaN,RC,687,NaN,XP_500977.1,NP_012965.3,GAP1
3,yarrowia_lipolytica,CACTCTGCCACCAAGTGGAT,NaN,NaN,F,626,NaN,XP_503263.1,NP_013406.1,MET17
4,yarrowia_lipolytica,TCTGCCACCAAGTGGATCGG,NaN,NaN,F,629,NaN,XP_503263.1,NP_013406.1,MET17
...,...,...,...,...,...,...,...,...,...,...
148,kluyveromyces_marxianus,TTGTACATACCTTTACCAGC,TTCTTCATACCTTTACCAGC,2.0,RC,2394,LYS2,XP_022678348.1,NP_009673.1,LYS2
149,kluyveromyces_marxianus,GTGAACATATCTCTTTGGAT,GTAAACATATCTCTTTGGAT,1.0,RC,2769,LYS2,XP_022678348.1,NP_009673.1,LYS2
150,kluyveromyces_marxianus,GTGAACATATCTCTTTGGAT,GTGAACATGTCTCTTTGGAT,1.0,RC,2769,LYS2,XP_022678348.1,NP_009673.1,LYS2
151,kluyveromyces_marxianus,GTGAACATATCTCTTTGGAT,GTGAAGATATCTCTTTGGAT,1.0,RC,2769,LYS2,XP_022678348.1,NP_009673.1,LYS2
